In [66]:
import pandas as pd
bicis = pd.read_excel("ny_citibikes_raw.xlsx")

# 1. Exploración del conjunto de datos
  - ¿Qué información contiene cada columna?
  

In [67]:
bicis.head()

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
0,2017-01-01 00:38:00,2017-01-01 01:03:00,3194,McGinley Square,3271,Danforth Light Rail,24668,Subscriber,1961,60,55-64,1513,25,1,Winter,10,Sunday
1,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
2,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
3,2017-01-01 01:56:00,2017-01-01 02:00:00,3186,Grove St PATH,3270,Jersey & 6th St,24604,Subscriber,1970,51,45-54,258,4,1,Winter,10,Sunday
4,2017-01-01 02:12:00,2017-01-01 02:23:00,3270,Jersey & 6th St,3206,Hilltop,24641,Subscriber,1978,43,35-44,663,11,1,Winter,10,Sunday


Esta tabla tiene una cantidad de 17 columnas:
- <u>Start Time:</u> Dice el momento donde empezó el viaje.
- <u>Stop Time:</u> Dice el momento donde acabó el viaje.
- <u>Start Station ID:</u> Da el ID de la estación donde comenzó el viaje.
- <u>Start Station Name:</u> Da el nombre de la estación donde comenzó el viaje.
- <u>End Station ID:</u> Da el ID de la estación donde acabó el viaje.
- <u>End Station Name:</u> Da el nombre de la estación donde acabó el viaje. 
- <u>Bike ID:</u> Da el ID de la bici que se utilizó en viaje.
- <u>User Type:</u> Da el tipo de usuario que realizó el viaje.
- <u>Birth Year:</u> Da el año de nacimiento del usuario.
- <u>Age:</u> Da la edad del usuario.
- <u>Age Groups:</u> Da el grupo de edad al que pertenece el usuario.
- <u>Trip Duration:</u> Tiempo en segundos que dura el viaje.
- <u>Trip_Duration_in_min:</u> Tiempo en minutos que dura el viaje.
- <u>Month:</u> Mes en el que realizó el viaje.
- <u>Season:</u> Estación climatica en el que realizó el viaje.
- <u>Temperature:</u> Temperatura con la que realizó el viaje.
- <u>Weekday:</u> Día de la semana en el que realizó el viaje.

- ¿Hay valores faltantes o duplicados?

Si los cuales pueden afectar a los resultados dando valores falseados, una vez encontrados se eliminarán para que no afecten los resultados.

In [68]:
print(f'Número de nulos: {bicis.isnull().sum()}')

print(f'Número de Duplicados: {bicis.duplicated().sum()}')


Número de nulos: Start Time              0
Stop Time               0
Start Station ID        0
Start Station Name      0
End Station ID          0
End Station Name        1
Bike ID                 0
User Type               0
Birth Year              0
Age                     0
Age Groups              0
Trip Duration           0
Trip_Duration_in_min    0
Month                   0
Season                  0
Temperature             0
Weekday                 0
dtype: int64
Número de Duplicados: 3555


Eliminamos valores duplicados para que no fecten a los resultados.

In [69]:
bicis= bicis.drop_duplicates()

- ¿Cuál es el lapso de tiempo total de los viajes?

Se puede ver que el més de febrero es el que más tiempo duran los viajes, debido a que puede ser que aunque sea en invierno sea cuando mejor tiempo hace.

In [93]:
bicis["Duracion Tiempo"]= pd.to_datetime(bicis["Stop Time"]) - pd.to_datetime(bicis["Start Time"])

bicis["Mes"] = bicis["Start Time"].dt.month

media_mes = bicis.groupby("Mes")["Duracion Tiempo"].mean()

media_mes

Mes
1   0 days 00:08:51.595583459
2   0 days 00:10:03.099762470
3   0 days 00:09:44.259827153
Name: Duracion Tiempo, dtype: timedelta64[ns]

# 2. Estadísticas básicas
  - ¿Cuál es la duración media del viaje (en minutos)?

  Se puede observar que son viajes cortos, ya que el poco tiempo que duran serán por que recorren pequeñas distancias.
  



In [72]:
print(f'Media: {bicis["Duracion Tiempo"].mean().total_seconds() / 60} minutos')

Media: 9.591154633333334 minutos


- ¿Cuál es la duración mínima y máxima?

El viaje más corto es de un minuto lo cual indica que fue una distáncia muy pequeña.

El viaje más largo es de 4 días esto indíca que debe de haber un error.
 

In [73]:
print(f'Mínimo: {bicis["Duracion Tiempo"].min()}')

print(f'Máximo: {bicis["Duracion Tiempo"].max()}')

Mínimo: 0 days 00:01:00
Máximo: 4 days 12:35:00


 - ¿Cuáles son las estaciones de inicio y fin más comunes?

 La parada de inicio más utilizada es la 3186 y la parada de fin más utilizada es la 3186, esto nos puede decir que estas paradas están en núcleos urbanos muy masificados y con mucho tráfico de personas.

 Puede ser que la primera parada sea una zona residencial y la parafa final una zona donde hay oficinas, o puede ser viceversa.

In [105]:
print(f'Início: {bicis["Start Station ID"].value_counts().head(1)} \n')
print(f'Fin: {bicis["End Station ID"].value_counts().head(1)}')

Início: Start Station ID
3186    2115
Name: count, dtype: int64 

Fin: End Station ID
3186    2743
Name: count, dtype: int64


# 3. Usuarios y datos demográficos
  - ¿Cuántas bicicletas únicas se utilizaron?

  Hay 500 bicicletas, esto nos sirbe para saber el número de bicicletas en circulación y hacerles un seguimiento para saber cuando hacer sus mantenimientos. 

In [75]:
print(f'Numero de bícis únicas: {len(bicis["Bike ID"].unique())}')

Numero de bícis únicas: 500


- ¿Cuál es la proporción de cada tipo de usuario (suscriptor vs. cliente)?

Muestra que hay más cantidad de subcriptores que de clientes, en caso de ser al contrario podría ser de ayuda mejorar las condiciones del usuario subcriptor para así incentivar a más gente y tener más ganancias.

In [76]:
print(f'Cantidad Subscriptores: {bicis["User Type"].value_counts(normalize=True)["Subscriber"]}')

print(f'Cantidad One-time user: {bicis["User Type"].value_counts(normalize=True)["One-time user"]}')

Cantidad Subscriptores: 0.9811219946571683
Cantidad One-time user: 0.0188780053428317



  - ​​¿Cuál es la distribución por edades de los usuarios? 
  Tenemos los grupod de:
  - 18-24
  - 25-34  
  - 35-44
  - 45-54
  - 55-64
  - 65-74
  - 75+

  - ¿Qué grupo de edad utiliza más el servicio?
  
  El grupo de edad que más utiliza el servicio es 35-44 ya que según la piramide de natalidad son el grupo que más ancho es, además de eso suelen estar más concienciados con la salud y actividad física.

In [104]:
print(f'Gupo edades: {bicis["Age Groups"].unique()}')

print(f'{bicis.groupby("Age Groups")['Age'].count()}')

Gupo edades: ['55-64' '25-34' '45-54' '35-44' '65-74' '18-24' '75+']
Age Groups
18-24      54
25-34    4002
35-44    7698
45-54    2973
55-64    1448
65-74     615
75+        55
Name: Age, dtype: int64


# 4. Análisis temporal
  - ¿Cómo varía el número de viajes según el día de la semana?

  Los primeros días de la semana son los que más actividad hay, esto puede deberse a que a principios de la semana la gente  empieza más motivada y a lo largo que abanza la semana la energía de la gente decae, los findes de semana son cuando menos hay ya que la gente está en su casa o se va fuera de la ciudad.




In [78]:
print(f'{bicis["Weekday"].value_counts(normalize=True)}')



Weekday
Wednesday    0.195963
Thursday     0.175304
Monday       0.149955
Tuesday      0.146037
Friday       0.145384
Saturday     0.094449
Sunday       0.092906
Name: proportion, dtype: float64


  - ¿Qué mes o temporada tiene la mayor cantidad de viajes?

In [79]:
print(f'{bicis["Season"].value_counts()} \n')

print(f'{bicis["Month"].value_counts()}')

Season
Winter    9671
Spring    7174
Name: count, dtype: int64 

Month
3    7174
2    5052
1    4619
Name: count, dtype: int64


 - ¿A qué hora del día comienzan la mayoría de los viajes?

In [80]:
bicis["Hora"] = bicis["Start Time"].dt.hour

hora_mas_frecuente = bicis["Hora"].value_counts().idxmax()
print(f"La hora con más viajes es: {hora_mas_frecuente}:00")

La hora con más viajes es: 8:00


# 5. Análisis geográfico
  - ¿Qué pares de estaciones (inicio → fin) aparecen con mayor frecuencia?

In [81]:
bicis[["Start Station ID","End Station ID"]].value_counts().head()

Start Station ID  End Station ID
3203              3186              401
3267              3183              366
3279              3186              293
3183              3267              290
3186              3203              273
Name: count, dtype: int64


  - ¿Hay alguna estación que aparezca solo como estación de inicio o solo como estación de fin?

In [82]:
bicis[["Start Station ID","End Station ID"]].value_counts().isna()
bicis[["End Station ID","Start Station ID"]].value_counts().isna()

End Station ID  Start Station ID
3186            3203                False
3183            3267                False
3186            3279                False
3267            3183                False
3203            3186                False
                                    ...  
3267            3207                False
                3195                False
                3191                False
3225            3277                False
3281            3281                False
Name: count, Length: 1154, dtype: bool

# 6. Temperatura y duración
  - ¿Existe alguna relación visible entre la temperatura y la duración del viaje?



In [83]:
bicis[["Trip Duration","Temperature"]].value_counts().head()


Trip Duration  Temperature
240            16             19
239            16             17
241            14             16
260            14             15
235            16             15
Name: count, dtype: int64

  - ¿Cómo varía la duración media del viaje según la temporada?

In [84]:
bicis.groupby("Season")["Trip Duration"].mean()

Season
Spring    583.219961
Winter    568.387240
Name: Trip Duration, dtype: float64

# 7. Resumen e interpretación
 - Escriba un breve resumen (5-10 líneas) de sus hallazgos.
 - Mencione patrones, anomalías o tendencias interesantes que haya observado.